<a href="https://colab.research.google.com/github/laurenchlebove/CS381/blob/main/TextProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Text Processing</h2>

1. Extract the numbers
2. Extract the names
3. Rearrange the vector so that all elements conform to the standard “firstname lastname”, preserving any titles (e.g., “Rev.”, “Dr.”, etc) or middle/second names.
4. Construct a logical vector indicating whether a character has a title (i.e., Rev. and Dr.)
5. Construct a logical vector indicating whether a character has a middle/second name.
6. Consider the HTML string \<title\>+++BREAKING NEWS+++\<title\>. We would like to extract the first HTML tag (i.e., “\<title\>”). To do so we write the regular expression “<.+>”. Explain why this fails and correct the expression.

7. Consider the string “(5-3)^2=5^2-2*5*3+3^2”. We would like to extract the equation in its entirety from the string. To do so we write the regular expression “[^0-9=+*()]+”. Explain why this fails and correct the expression.

In [22]:
import numpy as np
import pandas as pd
import re

def main():

  curr_data = ("555-1239Dr. Bernard Lander(636) 555-0113Hollingdorp, Donnatella555-6542Fitzgerald, " +
  "F. Scott555 8904Rev. Martin Luther King636-555-3226Snodgrass, Theodore5553642Carlamina Scarfoni")

  num_list = []
  name_list = []
  last_switch_index = 0
  isName = False

# Extract the names and numbers

  for i in range(len(curr_data)-1):
    if (isPartOfName(curr_data[i], isName)) and (not isPartOfName(curr_data[i+1], isName)):
      name_list.append(curr_data[last_switch_index:i+1])
      last_switch_index = i+1
      isName = not isName
    if (not isPartOfName(curr_data[i], isName)) and (isPartOfName(curr_data[i+1], isName)):
      num_list.append(curr_data[last_switch_index:i+1])
      last_switch_index = i+1
      isName = not isName

  if isName:
    name_list.append(curr_data[last_switch_index:])
  else:
    num_list.append(curr_data[last_switch_index:])


# map each string in name_list to commaLoc, and then use it to format names like "firstname lastname"

  name_list_locs = list(map(commaLoc, name_list))
  corrected_name_list = []

  for i in range(len(name_list)):
    if not name_list_locs[i] == "none":
      corrected_name_list.append(name_switcher(name_list[i], int(name_list_locs[i])))
    else:
      corrected_name_list.append(name_list[i])

  name_list = corrected_name_list
  
  print(name_list)


# Construct a logical vector indicating whether a character has a title (i.e., Rev. and Dr.)

  has_title_vector = [hasPeriod(x) for x in name_list]

  print(has_title_vector)
  

# Construct a logical vector indicating whether a character has a middle/second name.

  has_second_name_vector = []

  for i in range(len(name_list)):
    if has_title_vector[i]:
      has_second_name_vector.append((len(re.findall("\s", name_list[i]))) >= 3)
    else:
      has_second_name_vector.append((len(re.findall("\s", name_list[i]))) >= 2)
  
  print(has_second_name_vector)




# Given a string x, the function will return the index location of a comma character within x as a
# string value. If no such character exists in x, then it will return "None"
def commaLoc(x):
  for i in range(len(x)):
    if x[i] == ",":
      return str(i)
  return "none"



# Given the string x and integer position of comma, returns the string with the firstname and lastname
# in correct order and without unnecessary commas or spaces
def name_switcher(x, i):
  return (x[i+2:] + " " + x[:i])



# Given string x, returns True if string contains "Rev." or "Dr." and False if not
def hasPeriod(x):
  if re.search("Rev\.|Dr\.", x) == None:
    return False
  else:
    return True



# Given a string character x and a boolean value denoting whether or not the character in question is
# inside a person's name or phone number, the function will return True if the character is part of
# the name data, and false if it is not (and therefore part of the phone number data)
def isPartOfName(x, bool):
  return (str(x).isalpha() or x == "." or x == ",") or (x == " " and bool)




if __name__ == '__main__':
  main()

['Dr. Bernard Lander', 'Donnatella Hollingdorp', 'F. Scott Fitzgerald', 'Rev. Martin Luther King', 'Theodore Snodgrass', 'Carlamina Scarfoni']
[True, False, False, True, False, False]
[False, False, True, True, False, False]


Consider the HTML string \<title\>+++BREAKING NEWS+++\<title\>. We would like to extract the first HTML tag (i.e., “\<title\>”). To do so we write the regular expression “<.+>”. Explain why this fails and correct the expression.

In a regex, the "." signifies any character, and the "+" signifies one or more occurences of a character. So, the expression "<.+>" would look for a "<" character, and then one or more of any character at all, and then ending with a ">" character. This issue is ">" is considered to be one of the "any characters" that can be repeated within the ".+" portion of the regex, and it is only when the ">" in the last HTML tag is recognized that the regex is considered complete. This isn't what we want, since we only aim to extract the first HTML tag.

To fix this, we can change the expression to "<[^>]+>" - instead of allowing any character, as "." does, we use "[^>]" to allow any character besides ">", because we want the first instance of ">" to be recognizes as the end of the string fragment we're looking for.

Consider the string “(5-3)^2=5^2-2\*5\*3+3^2”. We would like to extract the equation in its entirety from the string. To do so we write the regular expression “[^0-9=+*()]+”. Explain why this fails and correct the expression.

The expression "[^0-9=+*()]+" would allow multiple "=" signs to be present in the string, which is not permissible in an equation. The regex ".+=.+" would instead allow only one "=" and all numbers and characters in the equation before and after the "=".